In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
# importing libraries

import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import matplotlib.pyplot as plt
from glob import glob

In [ ]:
image_set = 'C:/Users/Bryan/Documents/Semester 4/CompBio/Codes/Datasets'

In [ ]:
SIZE_X = SIZE_Y = 224

datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)

train_set = datagen.flow_from_directory(
    image_set,
    class_mode="categorical",
    target_size=(SIZE_X, SIZE_Y),
    color_mode="rgb",
    batch_size=32,
    shuffle=True, 
    subset='training',
    seed=42
)

validate_set = datagen.flow_from_directory(
    image_set,
    class_mode="categorical",
    target_size=(SIZE_X, SIZE_Y),
    color_mode="rgb",
    batch_size=32,
    shuffle=False,
    subset='validation',
    seed=42
)

In [ ]:
IMAGE_SIZE = [224, 224]

In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
flatten = Flatten()(vgg.output)
dense = Dense(256, activation = 'relu')(flatten)
dense = Dense(128, activation = 'relu')(dense)
prediction = Dense(3, activation = 'softmax')(dense)

In [ ]:
# creating a model
model = Model(inputs = vgg.input, outputs = prediction)

In [ ]:
model.summary()

In [ ]:
# output dari cell ini di clear karena filenya terlalu besar.

from tensorflow.keras.utils import plot_model

# Plot the model architecture
plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)

In [ ]:
# COMPILING THE MODEL
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
# executing the model
history = model.fit(train_set, validation_data = validate_set, epochs = 30, verbose = 1)

In [ ]:
# plotting the loss
plt.plot(history.history['loss'],label = 'train_loss')
plt.plot(history.history['val_loss'], label = 'testing_loss')
plt.title('loss')
plt.legend()
plt.show()

In [ ]:
# Both Validation and Training accuracy is shown here
plt.plot(history.history['accuracy'], label='training_accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.title('Accuracy')
plt.legend()
plt.show()

In [ ]:
# CHECKING THE CONFUSION MATRIX
from sklearn.metrics import classification_report, confusion_matrix, f1_score

Y_pred = model.predict(validate_set)
y_pred = np.argmax(Y_pred, axis = 1)
print('Confusion Matrix')
conf_matrix = confusion_matrix(validate_set.classes, y_pred)
print(conf_matrix)
print('Classification Report')
target_names = ['aca', 'n', 'scc']
print(classification_report(validate_set.classes, y_pred, target_names=target_names))

In [ ]:
# PLOTTING THE HEATMAP FOR CONFUSION MATRIX
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()